In [6]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"  # UYARILARI KAPATMA


In [7]:
model = keras.models.load_model(filepath="Kayıt/ilk eğitim vgg16 efd8f030/vgg_16_-saved-model-28-acc-0.51.hdf5")

In [8]:
from time import time
from tensorflow.python.keras.callbacks import TensorBoard, History
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
import pandas as pd

SIZE = 224
EPOCH = 50
BATCH = 32
train_directory = "images/train/"
valid_directory = "images/valid/"
test_directory = "images/test/"
data_aug = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)
validation_generator = data_aug.flow_from_directory(directory=valid_directory,
                                                    batch_size=BATCH,
                                                    shuffle=True,
                                                    class_mode="categorical",
                                                    color_mode="rgb",
                                                    target_size=(SIZE, SIZE),
                                                    interpolation="nearest",
                                                    seed=42
                                                    )
test_generator = data_aug.flow_from_directory(directory=test_directory,
                                              batch_size=1,
                                              shuffle=False,
                                              class_mode="categorical",
                                              color_mode="rgb",
                                              target_size=(SIZE, SIZE),
                                              seed=42
                                              )

Step_size_valid = validation_generator.n // validation_generator.batch_size
Step_size_test = test_generator.n // test_generator.batch_size


Found 777 images belonging to 37 classes.
Found 777 images belonging to 37 classes.


In [9]:
score, acc = model.evaluate_generator(generator=validation_generator,
                                      steps=Step_size_valid,

                                      )
test_generator.reset()

pred = model.predict_generator(generator=test_generator,
                               steps=Step_size_test,
                               verbose=1

                               )
predicted_class_indces = np.argmax(pred, axis=1)

777/777 [==============================] - 30s 38ms/step


In [10]:
labels = test_generator.class_indices
labels = dict((v, k) for k, v in labels.items())
predictions = [labels[k] for k in predicted_class_indces]

filenames = test_generator.filenames
results = pd.DataFrame({"Filename": filenames,
                        "Predictions": predictions})
results.to_csv("results.csv", index=False)
print("Test score ", score)
print("Test acc", acc)
results

Test score  1.6911402891079585
Test acc 0.4908854


,Filename,Predictions
0,Abyssinian/Abyssinian_1.jpg,Abyssinian
1,Abyssinian/Abyssinian_10.jpg,miniature_pinscher
2,Abyssinian/Abyssinian_11.jpg,Abyssinian
3,Abyssinian/Abyssinian_12.jpg,Russian_Blue
4,Abyssinian/Abyssinian_13.jpg,Abyssinian
...,...,...
772,yorkshire_terrier/yorkshire_terrier_4.jpg,yorkshire_terrier
773,yorkshire_terrier/yorkshire_terrier_5.jpg,havanese
774,yorkshire_terrier/yorkshire_terrier_6.jpg,scottish_terrier
775,yorkshire_terrier/yorkshire_terrier_7.jpg,scottish_terrier
